<a href="https://colab.research.google.com/github/Puurun/fastai_code_example/blob/main/titanic_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install fastai -Uqq

     |████████████████████████████████| 188 kB 4.3 MB/s 
     |████████████████████████████████| 60 kB 5.9 MB/s 


In [4]:
!mkdir ~/.kaggle
!cp /'content/drive/MyDrive/Colab Notebooks'/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 23.1MB/s]


In [6]:
!unzip titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [15]:
import pandas as pd
from fastai.tabular.all import *

In [40]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df[df['Survived']==2]['Survived'] = 0

KeyError: ignored

In [10]:
cat_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
reg_features = ['Age']
dep_var = 'Survived'

In [13]:
s = 0.8
cut = int(len(df)*s)
splits = list(range(cut)), list(range(cut, len(df)))

In [30]:
tdls = TabularPandas(df, procs=[Categorify, FillMissing, Normalize], cat_names=cat_features, cont_names=reg_features, y_names=dep_var, splits=splits)
tdls = tdls.dataloaders(64)

(tensor([[3, 2, 2, 1, 3, 1],
         [2, 2, 1, 1, 3, 1],
         [3, 2, 1, 1, 3, 1],
         [3, 1, 1, 3, 2, 2],
         [2, 2, 2, 1, 3, 1],
         [1, 1, 2, 1, 1, 1],
         [2, 2, 1, 1, 3, 1],
         [1, 2, 1, 3, 1, 1],
         [3, 2, 1, 1, 3, 1],
         [2, 2, 1, 1, 1, 1],
         [2, 1, 2, 1, 3, 1],
         [3, 2, 1, 1, 3, 2],
         [3, 1, 1, 2, 3, 1],
         [3, 2, 1, 1, 3, 1],
         [3, 1, 4, 2, 3, 2],
         [1, 2, 1, 1, 1, 1],
         [3, 2, 1, 1, 3, 2],
         [1, 2, 1, 1, 1, 1],
         [2, 2, 2, 1, 1, 1],
         [3, 2, 2, 1, 3, 1],
         [3, 2, 3, 1, 3, 1],
         [3, 2, 1, 1, 3, 1],
         [2, 2, 1, 1, 3, 1],
         [3, 1, 2, 2, 3, 1],
         [1, 2, 1, 1, 1, 1],
         [3, 2, 2, 1, 3, 1],
         [3, 2, 1, 1, 3, 1],
         [3, 2, 1, 1, 3, 2],
         [3, 2, 1, 1, 3, 1],
         [1, 2, 1, 1, 3, 2],
         [1, 1, 1, 1, 1, 1],
         [3, 2, 2, 2, 3, 1],
         [2, 1, 1, 1, 3, 1],
         [2, 1, 2, 1, 1, 1],
         [3, 2

In [41]:
tlearn = tabular_learner(tdls, [200, 100], n_out=2, loss_func=CrossEntropyLossFlat())

In [42]:
tlearn.fit_one_cycle(20)

epoch,train_loss,valid_loss,time
0,0.676259,0.655719,00:00
1,0.654438,0.644053,00:00
2,0.611249,0.639818,00:00
3,0.566262,0.636807,00:00
4,0.541788,0.585199,00:00
5,0.517251,0.523347,00:00
6,0.495158,0.528937,00:00
7,0.482289,0.433503,00:00
8,0.465119,0.369441,00:00
9,0.452117,0.350337,00:00


In [47]:
pred = [tlearn.predict(df_test.iloc[i])[1] for i in range(len(df_test))]

In [ ]:
preds = [p.item() for p in pred]
preds = [1 if p==1 else 2 for p in pred]

In [60]:
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': preds})
output.to_csv('submission.csv', index=False)

In [61]:
output

,PassengerId,Survived
0,892,2
1,893,2
2,894,2
3,895,2
4,896,2
...,...,...
413,1305,2
414,1306,1
415,1307,2
416,1308,2
